# Probando por separado

In [1]:
from cmp.evaluation import evaluate_reverse_parse

from engine.lexer.hulk_lexer import HulkLexer 
from engine.language.tokens_type import hulk_tokens
from engine.language.grammar import G
from engine.parser.hulk_parser import HulkParser
import dill
import sys

try:
    with open('engine/lexer/hulk_lexer.pkl', 'rb') as hulk_lexer_dp:
        hulk_lexer = dill.load(hulk_lexer_dp)
except:
    sys.setrecursionlimit(10000)
    hulk_lexer = HulkLexer(hulk_tokens, G.EOF)
    with open('engine/lexer/hulk_lexer.pkl', 'wb') as hulk_lexer_dp:
        dill.dump(hulk_lexer, hulk_lexer_dp)

In [2]:

try:
    with open('engine/parser/hulk_parser.pkl', 'rb') as hulk_parser_dp:
        hulk_parser = dill.load(hulk_parser_dp)
except:
    sys.setrecursionlimit(10000)
    hulk_parser = HulkParser()
    with open('engine/parser/hulk_parser.pkl', 'wb') as hulk_parser_dp:
        dill.dump(hulk_parser, hulk_parser_dp)

print(hulk_parser.tokens_terminals_map)

{'$': '$', <TokenType.EOF: 56>: '$', '$': '$', <TokenType.OPAR: 5>: '(', <TokenType.CPAR: 6>: ')', <TokenType.OPCOR: 9>: '[', <TokenType.CLCOR: 10>: ']', <TokenType.OPCUR: 7>: '{', <TokenType.CLCUR: 8>: '}', <TokenType.COMMA: 1>: ',', <TokenType.DOT: 2>: '.', <TokenType.COLON: 3>: ':', <TokenType.SEMI: 4>: ';', <TokenType.ARROW: 49>: '=>', <TokenType.DOUBLE_OR: 54>: '||', <TokenType.EQUAL: 26>: '=', <TokenType.DESTRUCTIVE: 55>: ':=', <TokenType.IDX: 11>: 'id', <TokenType.STRING: 12>: 'string', <TokenType.NUM: 13>: 'number', <TokenType.BOOLEAN: 14>: 'boolean', <TokenType.PLUS: 15>: '+', <TokenType.MINUS: 16>: '-', <TokenType.STAR: 17>: '*', <TokenType.DIV: 18>: '/', <TokenType.MOD: 19>: '%', <TokenType.POT: 20>: '^', <TokenType.POW2: 48>: '**', <TokenType.AND: 21>: '&', <TokenType.OR: 22>: '|', <TokenType.NOT: 23>: '!', <TokenType.CONCAT: 24>: '@', <TokenType.CONCAT_SP: 25>: '@@', <TokenType.EQUAL_EQUAL: 50>: '==', <TokenType.NOT_EQUAL: 51>: '!=', <TokenType.LESS_EQUAL: 53>: '<=', <Toke

In [3]:
from tests.test_cases import test_cases

tokens = []
right_parses = []
operations = []
asts = []

for input in test_cases:
    token = hulk_lexer(input)
    print(token)
    tokens.append(token)

    right_parse, operation = hulk_parser(token)
    right_parses.append(right_parse)
    operations.append(operation)

    ast = evaluate_reverse_parse(right_parse, operation, token)
    asts.append(ast)

    print(token)
    print(right_parse)
    print(operation)
    print(ast)
    print("="*100)


[TokenType.LET: let, TokenType.IDX: number, TokenType.EQUAL: =, TokenType.NUM: 42, TokenType.IN: in, TokenType.LET: let, TokenType.IDX: text, TokenType.EQUAL: =, TokenType.STRING: "The meaning of life is", TokenType.IN: in, TokenType.IDX: print, TokenType.OPAR: (, TokenType.IDX: text, TokenType.CONCAT: @, TokenType.IDX: number, TokenType.CPAR: ), TokenType.SEMI: ;, TokenType.EOF: $]
[TokenType.LET: let, TokenType.IDX: number, TokenType.EQUAL: =, TokenType.NUM: 42, TokenType.IN: in, TokenType.LET: let, TokenType.IDX: text, TokenType.EQUAL: =, TokenType.STRING: "The meaning of life is", TokenType.IN: in, TokenType.IDX: print, TokenType.OPAR: (, TokenType.IDX: text, TokenType.CONCAT: @, TokenType.IDX: number, TokenType.CPAR: ), TokenType.SEMI: ;, TokenType.EOF: $]
[<atomic_exp> -> number, <var_exp> -> <atomic_exp>, <not_exp> -> <var_exp>, <type_exp> -> <not_exp>, <pow_exp> -> <type_exp>, <factor> -> <pow_exp>, <term> -> <factor>, <arithmetic_exp> -> <term>, <concat_exp> -> <arithmetic_exp

In [4]:

from engine.semantic.type_builder_visitor import TypeBuilder
from engine.semantic.ast_printer import get_ast_printer
# from engine.semantic.type_checker_visitor import TypeChecker
from engine.semantic.type_collector_visitor import TypeCollector
from engine.semantic.type_inferer_visitor import TypeInferer
from engine.semantic.variable_collector_visitor import VarCollector


def semantic_analysis_pipeline(ast, debug=False):
    
    if debug:
        print('============== COLLECTING TYPES ===============')
    errors = []
    collector = TypeCollector(errors)
    collector.visit(ast)
    context = collector.context
    if debug:
        print('Errors: [')
        for error in errors:
            print('\t', error)
        print(']')
        print('Context:')
        print(context)
        print('=============== BUILDING TYPES ================')
    builder = TypeBuilder(context, errors)
    builder.visit(ast)
    if debug:
        print('Errors: [')
        for error in errors:
            print('\t', error)
        print(']')
        print('Context:')
        print(context)
        print('=============== CHECKING TYPES ================')
        print('---------------- COLLECTING VARIABLES ------------------')
    var_collector = VarCollector(context, errors)
    scope = var_collector.visit(ast)
    if debug:
        print('Errors: [')
        for error in errors:
            print('\t', error)
        print(']')
        print('Context:')
        print(context)
        print('Scope:')
        print(scope)
        print('---------------- INFERRING TYPES ------------------')
    # type_inferrer = TypeInferer(context, errors)
    # type_inferrer.visit(ast)

    # Check if there are any inference errors and change the types to ErrorType if there are
    # inference_errors = context.inference_errors() + scope.inference_errors()
    # errors.extend(inference_errors)
    # if debug:
    #     # print('Iterations: ' + str(type_inferrer.current_iteration))
    #     print('Errors: [')
    #     for error in errors:
    #         print('\t', error)
    #     print(']')
    #     print('Context:')
    #     print(context)
    #     print('Scope:')
    #     print(scope)
    #     print('---------------- CHECKING TYPES ------------------')
    # type_checker = TypeChecker(context, errors)
    # type_checker.visit(ast)
    # if debug:
    #     print('Errors: [')
    #     for error in errors:
    #         print('\t', error)
    #     print(']')
        # print('Context:')
        # print(context)
        # print('Scope:')
        # print(scope)
    return ast, errors, context, scope


In [5]:
print(semantic_analysis_pipeline(asts[6],debug=True))

============== COLLECTING TYPES ===============
Errors: [
]
Context:
{
	type Unknow {}
	
	type Object {
		[method] equals(other:Object): Boolean;
		[method] toString(): String;
	}
	
	type Number : Object {}
	
	type Boolean : Object {}
	
	type String : Object {
		[method] size(): Number;
		[method] next(): Boolean;
		[method] current(): String;
	}
	
	type Range : Object {
		[attrib] min : Number;
		[attrib] max : Number;
		[attrib] current : Number;
		[method] next(): Boolean;
		[method] current(): Number;
	}
	
	type Point {}
	
	type PolarPoint {}
	
	type Person {}
	
	type Knight {}
	
}
=============== BUILDING TYPES ================
visitando el tipo Point
el current type es Point
visitando un atributo x
el tipo de atributo asignado es Unknow
visitando un atributo y
el tipo de atributo asignado es Unknow
2
visitando un metodo setY
el parametro y de tipoUnknow annadido
el tipo de retorno del metodo es Unknow
visitando un metodo getY
el tipo de retorno del metodo es Unknow
visitando un m

In [6]:
print(semantic_analysis_pipeline(asts[7],debug=True))
from engine.semantic.ast_printer import get_ast_printer
printer = get_ast_printer()
print(printer(asts[7]))

============== COLLECTING TYPES ===============
Errors: [
]
Context:
{
	type Unknow {}
	
	type Object {
		[method] equals(other:Object): Boolean;
		[method] toString(): String;
	}
	
	type Number : Object {}
	
	type Boolean : Object {}
	
	type String : Object {
		[method] size(): Number;
		[method] next(): Boolean;
		[method] current(): String;
	}
	
	type Range : Object {
		[attrib] min : Number;
		[attrib] max : Number;
		[attrib] current : Number;
		[method] next(): Boolean;
		[method] current(): Number;
	}
	
	type PolarPoint {}
	
}
=============== BUILDING TYPES ================
visitando el tipo PolarPoint
el current type es PolarPoint
visitando un atributo x
el tipo de atributo asignado es Unknow
visitando un atributo strin
el tipo de atributo asignado es Unknow
visitando un atributo y
el tipo de atributo asignado es Unknow
3
visitando un metodo retornaphi
el tipo de retorno del metodo es Unknow
Errors: [
]
Context:
{
	type Unknow {}
	
	type Object {
		[method] equals(other:Object)

In [7]:
from cmp.cil import get_formatter
from engine.code_generation.visitor import HulkToCILVisitor

formatter = get_formatter()

def gencode_pipeline(ast, debug):
    ast, _, context, _ = semantic_analysis_pipeline(ast, debug=debug)
    print('============= TRANSFORMING TO CIL =============')
    hulk_to_cil = HulkToCILVisitor(context)
    cil_ast = hulk_to_cil.visit(ast)
    formatter = get_formatter()
    print(formatter(cil_ast))
    return ast, cil_ast

In [8]:
ast, cil_ast = gencode_pipeline(asts[1], debug=False) 

VarCollector Visitor
LetIn node
VarDeclaration Node
VarDeclaration Node
VarDeclaration Node
Var node a
Var node b
Var node b
Var node c
Var node b
Var node c
Var node a
============= TRANSFORMING TO CIL =============
buscando variable  a
buscando variable  b
buscando variable  c
<engine.language.ast_nodes.ExpressionBlockNode object at 0x0000024B00CDC310> aquiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
functionCall
{'a': 'local_main_at_Main_a_0', 'b': 'local_main_at_Main_b_1', 'c': 'local_main_at_Main_c_2', 'internal': 'local_main_at_Main_internal_3'}
{'a': 'local_main_at_Main_a_0', 'b': 'local_main_at_Main_b_1', 'c': 'local_main_at_Main_c_2', 'internal': 'local_main_at_Main_internal_3'}
functionCall
{'a': 'local_main_at_Main_a_0', 'b': 'local_main_at_Main_b_1', 'c': 'local_main_at_Main_c_2', 'internal': 'local_main_at_Main_internal_4'}
{'a': 'local_main_at_Main_a_0', 'b': 'local_main_at_Main_b_1', 'c': 'local_main_at_Main_c_2', 'internal': 'local_main_at_Main_internal_4'}
{'a': 'local_main_at_M